In [3]:
import numpy as np 
import pandas as pd 
import mf
from nfc.src.mlp import MLP as mlp
from scipy.sparse import csr_matrix
from mf import MF 
import os
import sys

In [2]:
DATA_DIR = '/afs/inf.ed.ac.uk/user/s18/s1877727/recommendations/ml-20m'
raw_data = pd.read_csv(os.path.join(DATA_DIR, 'ratings.csv'), header=0)
raw_data = raw_data[raw_data['rating'] > 3.5]

In [1]:
def get_count(tp, id):
    playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
    count = playcount_groupbyid.size()
    return count

def filter_triplets(tp, min_uc=5, min_sc=0):
    # Only keep the triplets for items which were clicked on by at least min_sc users. 
    if min_sc > 0:
        itemcount = get_count(tp, 'movieId')
        tp = tp[tp['movieId'].isin(itemcount.index[itemcount >= min_sc])]
    
    # Only keep the triplets for users who clicked on at least min_uc items
    # After doing this, some of the items will have less than min_uc users, but should only be a small proportion
    if min_uc > 0:
        usercount = get_count(tp, 'userId')
        tp = tp[tp['userId'].isin(usercount.index[usercount >= min_uc])]
    
    # Update both usercount and itemcount after filtering
    usercount, itemcount = get_count(tp, 'userId'), get_count(tp, 'movieId') 
    return tp, usercount, itemcount

In [4]:
#Only keep items that are clicked on by at least 5 users
raw_data, user_activity, item_popularity = filter_triplets(raw_data)
sparsity = 1. * raw_data.shape[0] / (user_activity.shape[0] * item_popularity.shape[0])

print("After filtering, there are %d watching events from %d users and %d movies (sparsity: %.3f%%)" % 
      (raw_data.shape[0], user_activity.shape[0], item_popularity.shape[0], sparsity * 100))

unique_uid = user_activity.index

np.random.seed(98765)
idx_perm = np.random.permutation(unique_uid.size)
unique_uid = unique_uid[idx_perm]

# create train/validation/test users
n_users = unique_uid.size
n_heldout_users = 10000

tr_users = unique_uid[:(n_users - n_heldout_users * 2)]
vd_users = unique_uid[(n_users - n_heldout_users * 2): (n_users - n_heldout_users)]
te_users = unique_uid[(n_users - n_heldout_users):]

After filtering, there are 9990682 watching events from 136677 users and 20720 movies (sparsity: 0.353%)


In [2]:
def split_train_test_proportion(data, test_prop=0.2):
    data_grouped_by_user = data.groupby('userId')
    tr_list, te_list = list(), list()

    np.random.seed(98765)

    for i, (_, group) in enumerate(data_grouped_by_user):
        n_items_u = len(group)

        if n_items_u >= 5:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        else:
            tr_list.append(group)

        if i % 1000 == 0:
            print("%d users sampled" % i)
            sys.stdout.flush()

    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)
    
    return data_tr, data_te

In [6]:
train_plays = raw_data.loc[raw_data['userId'].isin(tr_users)]

In [7]:
unique_sid = pd.unique(train_plays['movieId'])

In [8]:
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

In [9]:
pro_dir = os.path.join(DATA_DIR, 'pro_sg')
if not os.path.exists(pro_dir):
    os.makedirs(pro_dir)

with open(os.path.join(pro_dir, 'unique_sid.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)

In [10]:
vad_plays = raw_data.loc[raw_data['userId'].isin(vd_users)]
vad_plays = vad_plays.loc[vad_plays['movieId'].isin(unique_sid)]

In [11]:
vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays)

0 users sampled
1000 users sampled
2000 users sampled
3000 users sampled
4000 users sampled
5000 users sampled
6000 users sampled
7000 users sampled
8000 users sampled
9000 users sampled


In [12]:
test_plays = raw_data.loc[raw_data['userId'].isin(te_users)]
test_plays = test_plays.loc[test_plays['movieId'].isin(unique_sid)]

In [13]:
test_plays_tr, test_plays_te = split_train_test_proportion(test_plays)

0 users sampled
1000 users sampled
2000 users sampled
3000 users sampled
4000 users sampled
5000 users sampled
6000 users sampled
7000 users sampled
8000 users sampled
9000 users sampled


In [4]:
def numerize(tp):
    uid = map(lambda x: profile2id[x], tp['userId'])
    sid = map(lambda x: show2id[x], tp['movieId'])
    return pd.DataFrame(data={'uid': list(uid), 'sid': list(sid)}, columns=['uid', 'sid'])

In [15]:
train_data = numerize(train_plays)
train_data.to_csv(os.path.join(pro_dir, 'train.csv'), index=False)

In [16]:
vad_data_tr = numerize(vad_plays_tr)
vad_data_tr.to_csv(os.path.join(pro_dir, 'validation_tr.csv'), index=False)

In [17]:
vad_data_te = numerize(vad_plays_te)
vad_data_te.to_csv(os.path.join(pro_dir, 'validation_te.csv'), index=False)

In [18]:
test_data_tr = numerize(test_plays_tr)
test_data_tr.to_csv(os.path.join(pro_dir, 'test_tr.csv'), index=False)

In [19]:
test_data_te = numerize(test_plays_te)
test_data_te.to_csv(os.path.join(pro_dir, 'test_te.csv'), index=False)


In [5]:
# Headers and imports
import numpy as np 
import pandas as pd 
import mf
from nfc.src.mlp import MLP as mlp
from scipy.sparse import csr_matrix
from mf import MF 
import os
import sys

from pytorch.torchmf import BasePipeline,bpr_loss,BPRModule,PairwiseInteractions
from scipy.sparse import coo_matrix
import torch

train_data = pd.read_csv('ml-20m/pro_sg/train.csv')
test_data_tr = pd.read_csv('ml-20m/pro_sg/test_tr.csv')
users = 136677 
movies = 20720


In [6]:
users_train = train_data.uid.values
movies_train = train_data.sid.values
data_train = np.ones((users_train.shape[0]))
coo_train = coo_matrix((data_train, (users_train,movies_train)),shape=(users, movies))

#Dimensions of the coo_matrix (Users x Movies)
n_test_users = test_data_tr.uid.unique()
n_test_movies =  test_data_tr.sid.unique()

users_test = test_data_tr.uid.values
movies_test = test_data_tr.sid.values
data_test = np.ones((users_test.shape[0]))
coo_test = coo_matrix((data_test, (users_test,movies_test)),shape=(users, movies))

In [7]:
pipeline = BasePipeline(train= coo_train, test=coo_test, verbose=True,
                        batch_size=1024, num_workers=4,
                        n_factors=20, weight_decay=0,
                        dropout_p=0., lr=.2, sparse=True,
                        optimizer=torch.optim.SGD, n_epochs=40,
                        random_seed=2017, loss_function=bpr_loss,
                        model=BPRModule, hogwild=True,
                        interaction_class=PairwiseInteractions,
                        eval_metrics=('auc', 'patk'))


In [8]:
pipeline.fit()